In [1]:
import psutil
import time

# Obtener estadísticas de uso
def obtener_metricas_sistema(host_id):
    # Uso de CPU (promedio de los últimos segundos)
    cpu_usage = psutil.cpu_percent(interval=1)
    
    # Uso de RAM
    mem = psutil.virtual_memory()
    ram_used_gb = round(mem.used / (1024**3), 2) # Conversión a GB
    ram_percent = mem.percent
    
    # Uso de disco (en el punto de montaje raíz)
    disk = psutil.disk_usage('/')
    disk_percent = disk.percent
    
    return {
        'host': host_id,
        'cpu_percent': cpu_usage,
        'ram_used_gb': ram_used_gb,
        'ram_percent': ram_percent,
        'disk_percent': disk_percent
    }

In [2]:
import influxdb_client
from influxdb_client.client.write_api import ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=hola"

client = None
try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    health = client.health()

    if health.status == "pass":
        print("Conexión exitosa")
    else:
        print("Conexión fallida")

except(InfluxDBError,NewConnectionError) as e:
    print("Error")
    print(e)

Verificando estado de salud de InfluxDB en http://influxdb2:8086...
Conexión exitosa


In [3]:
from influxdb_client import Point, WriteOptions
import time 

writeoptions = WriteOptions(
        batch_size = 500,
        flush_interval = 1000,
        write_type = "ASYNCHRONOUS"
    )

write_api = client.write_api(write_options=writeoptions)


try:
    while(True):
        metricas = obtener_metricas_sistema("servidor_01")

        p = Point("rendimiento_servidor").tag("host_id",metricas['host']).tag("entorno","producion") \
        .field("cpu_percent",metricas['cpu_percent']).field("ram_percent",metricas['ram_percent']).field("disk_percent",metricas['disk_percent'])

        write_api.write(bucket = "iot_data", org= "docs", record = p)

        print("Escribiendo...")
    
        time.sleep(1)

        
except KeyboardInterrupt:
    write_api.close()
    print("Fin de la ejecución")



Escribiendo...
Escribiendo...
Escribiendo...
Escribiendo...
Fin de la ejecución
